In [6]:
import numpy as np
import pandas as pd

from pathlib import Path
from tqdm import tqdm

import torchaudio
from sklearn.model_selection import train_test_split

import librosa
import IPython.display as ipd

import os
import sys

In [7]:
ravdess_directory_list = os.listdir('data/ravdess/Audio_Speech_Actors_01-24')

file_emotion = []
file_path = []
for dir in ravdess_directory_list:
    # as their are 20 different actors in our previous directory we need to extract files for each actor.
    actor = os.listdir(os.path.join('data', 'ravdess', 'Audio_Speech_Actors_01-24', dir))
    for file in actor:
        part = file.split('.')[0]
        part = part.split('-')
        # third part in each file represents the emotion associated to that file.
        file_emotion.append(int(part[2]))
        file_path.append('data/ravdess/Audio_Speech_Actors_01-24' + dir + '/' + file)

# dataframe for emotion of files
emotion_df = pd.DataFrame(file_emotion, columns=['Emotions'])

# dataframe for path of files.
path_df = pd.DataFrame(file_path, columns=['Path'])
Ravdess_df = pd.concat([emotion_df, path_df], axis=1)

# changing integers to actual emotions.
Ravdess_df.Emotions.replace({1:'neutral', 2:'calm', 3:'happy', 4:'sad', 5:'angry', 6:'fear', 7:'disgust', 8:'surprise'}, inplace=True)
Ravdess_df.head(20)

,Emotions,Path
0,neutral,data/ravdess/Audio_Speech_Actors_01-24Actor_01...
1,neutral,data/ravdess/Audio_Speech_Actors_01-24Actor_01...
2,neutral,data/ravdess/Audio_Speech_Actors_01-24Actor_01...
3,neutral,data/ravdess/Audio_Speech_Actors_01-24Actor_01...
4,calm,data/ravdess/Audio_Speech_Actors_01-24Actor_01...
5,calm,data/ravdess/Audio_Speech_Actors_01-24Actor_01...
6,calm,data/ravdess/Audio_Speech_Actors_01-24Actor_01...
7,calm,data/ravdess/Audio_Speech_Actors_01-24Actor_01...
8,calm,data/ravdess/Audio_Speech_Actors_01-24Actor_01...
9,calm,data/ravdess/Audio_Speech_Actors_01-24Actor_01...


In [ ]:
data = []
emotion_map = {
    '01': 'neutral',
    '02': 'calm',
    '03': 'happy',
    '04': 'sad',
    '05': 'angry',
    '06': 'fear',
    '07': 'disgust',
    '08': 'surprise'
}

for path in tqdm(Path("data/ravdess/Audio_Speech_Actors_01-24").glob("**/*.wav")):
    
    label = (str(path).split('.')[0]).split('/')[-1]
    label = label.split('-')[3] #why not 2?
    
    try:
        # There are some broken files
        s = torchaudio.load(path)
        data.append({
            "emotion": emotion_map[label],
            "path": path
        })
    except Exception as e:
        # print(str(path), e)
        pass
df = pd.DataFrame(data)
df.head(20)

1440it [00:02, 537.85it/s]


,emotion,path
0,neutral,data\ravdess\Audio_Speech_Actors_01-24\Actor_0...
1,neutral,data\ravdess\Audio_Speech_Actors_01-24\Actor_0...
2,neutral,data\ravdess\Audio_Speech_Actors_01-24\Actor_0...
3,neutral,data\ravdess\Audio_Speech_Actors_01-24\Actor_0...
4,calm,data\ravdess\Audio_Speech_Actors_01-24\Actor_0...
5,calm,data\ravdess\Audio_Speech_Actors_01-24\Actor_0...
6,calm,data\ravdess\Audio_Speech_Actors_01-24\Actor_0...
7,calm,data\ravdess\Audio_Speech_Actors_01-24\Actor_0...
8,calm,data\ravdess\Audio_Speech_Actors_01-24\Actor_0...
9,calm,data\ravdess\Audio_Speech_Actors_01-24\Actor_0...


In [5]:
print("Labels: ", df["emotion"].unique())
print()
df.groupby("emotion").count()[["path"]]

Labels:  ['neutral' 'calm' 'happy' 'sad' 'angry' 'fear' 'disgust' 'surprise']



,path
emotion,
angry,192
calm,192
disgust,192
fear,192
happy,192
neutral,96
sad,192
surprise,192


In [ ]:
idx = np.random.randint(0, len(df))
sample = df.iloc[idx]
path = sample["path"]
label = sample["emotion"]


print(f"ID Location: {idx}")
print(f"      Label: {label}")
print()

speech, sr = torchaudio.load(path)
speech = speech[0].numpy().squeeze()
speech = librosa.resample(np.asarray(speech), sr, 16_000)
ipd.Audio(data=np.asarray(speech), autoplay=True, rate=16000)